# Shamir's Secret Sharing

L'obiettivo di questo notebook è mostrare il funzionamento dello Shamir's Secret Sharing.
Si tratta di un algoritmo crittografico che consente di suddividere un segreto in $N$ parti, chiamate quote (share), tali che:
- il segreto possa essere ricostruito solo se si possiedono almeno $K$ quote (soglia o threshold).
- con meno di $K$ quote non si ottengono informazioni utili sul segreto (grazie all'aritmetica modulare sui campi finiti).

## Preparazione

Importiamo le librerie necessarie e definiamo una classe `Share` che rappresenta una singola quota del segreto.

In [1]:
import secrets
import base64
from typing import List

from cryptography.hazmat.primitives.ciphers.aead import AESGCM
from cryptography.exceptions import InvalidTag

In [2]:
class Share:
    def __init__(
        self,
        x: int,
        y: int,
    ):
        self.x = x
        self.y = y

    def __repr__(self):
        return (
            f"Share(x={self.x}, y={self.y})"
        )

    def serialize(self) -> dict:
        return {
            "x": base64.b64encode(self.x.to_bytes((self.x.bit_length() + 7) // 8 or 1, "big")).decode(),
            "y": base64.b64encode(self.y.to_bytes((self.y.bit_length() + 7) // 8 or 1, "big")).decode()
        }

    @staticmethod
    def deserialize(data: dict) -> "Share":
        x_bytes = base64.b64decode(data["x"])
        y_bytes = base64.b64decode(data["y"])
        x = int.from_bytes(x_bytes, "big")
        y = int.from_bytes(y_bytes, "big")
        return Share(
            x=x,
            y=y,
        )

## Polinomi sui campi finiti

Utilizziamo un numero primo a 256 bit (NIST P-256) per l'aritmetica modulare e valutiamo i polinomi con il metodo di Horner.

In [3]:
PRIME: int = 0xffffffff00000001000000000000000000000000ffffffffffffffffffffffff

def eval_polynomial(x: int, coeffs: List[int], prime: int = PRIME) -> int:
    y = 0
    tmp = 1
    for coeff in coeffs:
        y = (y + coeff * tmp) % prime
        tmp = (tmp * x) % prime
    return y

## Suddivisione del segreto e cifratura del messaggio
Generiamo una chiave casuale, la usiamo per cifrare un messaggio, e poi la suddividiamo in più quote.

In [4]:
def split_secret(
    secret: int,
    shares_count: int,
    threshold: int
) -> List[Share]:
    poly_coeffs: List[int] = [secret] + [secrets.randbelow(PRIME - 1) + 1 for _ in range(threshold - 1)]

    shares: List[Share] = []
    for x in range(1, shares_count + 1):  # x starts from 1
        y: int = eval_polynomial(x, poly_coeffs, PRIME)
        shares.append(Share(x, y))

    return shares



In [ ]:
# example parameters
message = "My secret vault's key is 8347"
shares_count = 5
threshold = 3

key = secrets.token_bytes(32)
cipher = AESGCM(key)
nonce = secrets.token_bytes(12)
message_bytes = message.encode()
ciphertext = cipher.encrypt(nonce, message_bytes, None)
print(f"Ciphertext: {ciphertext}")

secret_int = int.from_bytes(key, "big")
shares: List[Share] = split_secret(secret_int, shares_count, threshold)

for share in shares:
    print(share)

serialized_shares = []
for share in shares:
    s = share.serialize()
    serialized_shares.append(s)

Ciphertext: b'\xb5\t\xc3\x19\x7f\xac3\xbe\x1e\xe6\xc69vBQ\xf0\x7f\x98@<\xdc\xef\xb8\xcf\xc8\x8d\x93\xaa-\x1f\xfb\xbcDl\xea\xf3\xea+\xf8\x90<\xc1\x87\xb1e'
Share(x=1, y=66893479876738493363665383929616020815086838301155644341685445868044924695828)
Share(x=2, y=93694531338731097724797961713766269125526011793131151341871404833667438794517)
Share(x=3, y=46642040950732683917560916795946083290442568047875701672671912724659076064076)
Share(x=4, y=41528097923099500704651696125563036839922650480679609529620600849886934358456)
Share(x=5, y=78352702255831548086070299702617129773966259091542874912717469209351013677657)


## Ricostruzione del segreto e decifratura del messaggio

Utilizzando almeno `threshold` quote, ricostruiamo il segreto tramite interpolazione di Lagrange.

In [6]:
def reconstruct_secret(shares: List[Share], prime: int) -> int:
    secret: int = 0
    n: int = len(shares)

    for i in range(n):
        xi, yi = shares[i].x, shares[i].y
        li: int = 1
        for j in range(n):
            if i != j:
                xj = shares[j].x
                numerator: int = (-xj) % prime
                denominator: int = (xi - xj) % prime
                li = (li * numerator * pow(denominator, -1, prime)) % prime
        secret = (secret + yi * li) % prime

    return secret

In [7]:
shuffled_shares = secrets.SystemRandom().sample(serialized_shares, threshold)
subset = [Share.deserialize(s) for s in shuffled_shares]

secret_int_reconstructed = reconstruct_secret(subset, PRIME)
key_reconstructed = secret_int_reconstructed.to_bytes(32, "big")

cipher2 = AESGCM(key_reconstructed)
try:
    plaintext = cipher2.decrypt(nonce, ciphertext, None)
    print("Decrypted message: " + plaintext.decode())
except InvalidTag as e:
    print("Decryption failed")

Decrypted message: My secret vault's key is 8347
